# Tutorial: Constructing a model in gemlib


This notebook demonstrates `gemlib`, a library for implementing infectious disease models.  It presents unified framework for expressing and simulating models, as well as automatic generation of probability functions used for parameter inference.

__Documentation__: https://gem-epidemics.gitlab.io/gemlib


## The epidemic model
We demonstrate the use of `gemlib` by constructing a discrete-time stochastic metapopulation model with a static connectivity network.

We define our population as a composition of 3 connected metapopulations (units), in which individuals are classified into states. Here, we have 3 _states_: $S$, $I$, and $R$ representing susceptible, infected, and recovered individuals respectively.We then have two transitions, $S \rightarrow I$ and $I \rightarrow R$. For each unit $i$ at time $t$ the infection rate ($S \rightarrow I$) is proportional to the number of infected individuals in unit $i$ and the sum of the infected individuals in all other units, weighted by a connectivity matrix.  The infection rate for unit $i$ at time $t$ is as follows:

\begin{equation}
\lambda^{\mathrm{SI}}_i(t, x_t)  = \frac{\beta_1  x_{it}^{I} + \beta_2 (\vec{C} \cdot \vec{x}_t^{I})}{\vec{n}}
\end{equation}

where $C$ is a zero-diagonal connectivity matrix between units, $\vec{n}$ is a vector of population sizes, and $\theta = \{\beta_1, \beta_2\}$ are parameters to be estimated.

We assume the transition rate from infected to removed ($I \rightarrow R$) is known, and constant across time and population. Denoted by:

\begin{equation}
\lambda^{\mathrm{IR}}_i(t, {x}_t) = \gamma\,{day}^{-1}
\end{equation}

where $\gamma = 1/10$.

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_probability.substrates.jax as tfp

from gemlib.distributions import DiscreteTimeStateTransitionModel

tfd = tfp.distributions

## Implementing the SIR model in `gemlib`

Having specified our _states_, the _transitions_ between states, and the _rates_ at which they occur, we can start to assemble a model for the disease process.  To do this, we will use `gemlib`'s `DiscreteTimeStateTransitionModel`, which embeds the above model into a discrete-time state transition stochastic process.

There are 3 key ingredients to implementing a model:

* The _incidence matrix_ which describes the state transition graph
* The _initial state_ matrix describing the global state of the epidemic at the beginning
* The _transition rate function_, a list of transition functions, which given the time and a global state, computes the transition rates for each individual.

In [ ]:
incidence_matrix = np.array(
    [  #  SI  IR
        [-1, 0],  # S
        [1, -1],  # I
        [0, 1],   # R
    ],
    dtype=np.float32,
)

In [ ]:
# Define the initial state for the 3 metapopulations.
initial_state = np.array(
    [  # S   I  R
        [99, 1, 0],
        [100, 0, 0],
        [100, 0, 0],
    ],
    dtype=np.float32,
)

In [ ]:
# Time parameters
initial_step = 0.0
time_delta = 1.0
num_steps = 100

In [ ]:
# Deine a static network of inter-metapopulation connectivity (MxM).
contact_network = jax.random.uniform(
    minval=0,
    maxval=10,
    shape=(initial_state.shape[0], initial_state.shape[0]),
    key=jax.random.key(0),
)

contact_network = jnp.fill_diagonal(
    contact_network, 0, inplace=False
)  # set intra-connectivity/diagonals to 0

In [ ]:
# Define model parameters
beta1 = 0.1
beta2 = 0.01
gamma = 1 / 10

In [ ]:
# Define the transition rate functions
def si_rate(_, state):
    return (
        beta1 * state[:, 1] + (beta2 * jnp.matvec(contact_network, state[:, 1]))
    ) / state.sum(axis=-1)


def ir_rate(_0, _1):
    return gamma

In [ ]:
# Instantiate the epidemic model
discrete_sir = DiscreteTimeStateTransitionModel(
    transition_rate_fn=[si_rate, ir_rate],
    incidence_matrix=incidence_matrix,
    initial_state=initial_state,
    initial_step=initial_step,
    time_delta=time_delta,
    num_steps=num_steps,
    name="sir",
)

## Simulating epidemic data
We can now simulate from the discrete_sir model, setting sample_shape=100 for 100 trajectories.

In [ ]:
# Simulate the epidemic trajectory 100 times
stoch_events = jax.jit(
    lambda: discrete_sir.sample(sample_shape=100, seed=jax.random.key(0))
)()

# Compute the states for each trajectory
stoch_states = discrete_sir.compute_state(stoch_events)

We now plot the epidemic trajectories given by the `DiscreteTimeStateTransitionModel` aggregated over the three metapopulations.

In [ ]:
# Plot the epidemic timeseries.
def plot_timeseries(stoch_states):
    cols = ["tab:blue", "tab:red", "tab:green"]
    labels = ["Susceptible", "Infected", "Recovered"]

    for states, col in enumerate(cols):
        plt.plot(
            stoch_states[:, :, :, states].sum(axis=2).T, color=col, alpha=0.2
        )
    for col, label in zip(cols, labels, strict=False):
        plt.plot([], [], color=col, label=label)

    plt.xlabel("Time")
    plt.ylabel("Individuals")
    plt.legend()
    plt.show()

In [ ]:
plot_timeseries(stoch_states)

## Computing the log-probability
We can now embed our model into an instance of TensorFlow Probability's JointDistributionCoroutineAutoBatched. Here, we specify the prior distributions for the parameters we wish to estimate and then instantiate the DiscreteTimeStateTransitionModel. This enables us to easily compute the log-probability of the model.

In [ ]:
@tfd.JointDistributionCoroutineAutoBatched
def model():
    # Priors
    beta1 = yield tfd.Gamma(
        concentration=0.5,
        rate=10.0,
        name="beta1",
    )
    beta2 = yield tfd.Gamma(
        concentration=0.2,
        rate=10.0,
        name="beta2",
    )

    # Epidemic model
    gamma = 1 / 10

    def si_rate(_, state):
        return (
            beta1 * state[:, 1]
            + (beta2 * jnp.matvec(contact_network, state[:, 1]))
        ) / state.sum(axis=-1)

    def ir_rate(_0, _1):
        return gamma

    sir_model = DiscreteTimeStateTransitionModel(
        transition_rate_fn=[si_rate, ir_rate],
        incidence_matrix=incidence_matrix,
        initial_state=initial_state,
        initial_step=initial_step,
        time_delta=time_delta,
        num_steps=num_steps,
        name="sir",
    )
    yield sir_model

This enables us to simulate an epidemic and compute the log-probability given beta1, beta2 and the simulated epidemic events.

In [ ]:
# simulate an epidemic
sir_sim = model.sample(seed=jax.random.key(1))

In [ ]:
# Compute the log-probability given beta1, beta2 and epidemic events
model.log_prob(beta1=0.05, beta2=0.005, sir=sir_sim.sir)